<a href="https://colab.research.google.com/github/D40848397/DW_Sasi_pub/blob/master/CMICheck1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")
#print("date and time =", dt_string)	

In [4]:
def addtoDB(filepath):
  df = pd.read_csv(filepath, skiprows=1, names=['table', 'scount', 'traget', 'Sep','x','tragetCount'], sep=r'\s+', engine='python') 
  return df

In [5]:
def targetClean(target, x):
  if (isinstance(target, int) or target.isdigit()):
    return int(target)
  else:
    if isinstance(target, str):
        if isinstance(x, int) and x > 0:
          return int(x)

In [6]:
#save to SQLite

import sqlite3
con = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/Sasi-Dev/CMIAdminLog.db")

def storeDB(df):
  df.to_sql(name='syncCheck', con=con, if_exists='append')
  return

In [7]:
directory = r'/content/drive/My Drive/CMICompareLogs'
schoollist =['BNR']
for entry in os.scandir(directory):
  #print(entry.path)
  if (entry.path.endswith(":00")):
      path = entry.path
      if any(ele in path for ele in schoollist):
        
        with open(entry.path, 'r') as f:
          ftimestamp= f.readline()
          ftimestamp= ftimestamp[:17]
          insti = path[path.rfind('/')+1:path.rfind('BNR_')-1]
          #print(ftimestamp)
          #print(entry.path)
          df = addtoDB(entry.path)
          df['traget'] = df['Sep'] 
          df=df.drop(columns=['Sep','tragetCount'])
          df = df.fillna(value=0)
          df=df[df.table != 0]

          badx= ['Differences','========','compilation']
          df = df[~df.x.isin(badx)]

          badt= [0,'<']
          df = df[~df.table.isin(badt)]

          df['x'] = pd.to_numeric(df['x'])
          df['traget'] = df.apply(lambda x: targetClean(x['traget'],x['x']),axis=1) 
          df=df.drop(columns=['x'])
          df['runTimeStamp']= ftimestamp
          df['institution']=insti
          df['table']= df['table'].str.replace(',','')
          df['table']= df['table'].str.replace(' ','')
          df=df.drop_duplicates()
          try:
            storeDB(df)
          except:
            df.head(3)
          
          # move burned files to archive 
          head_tail = os.path.split(path) 
          dirname = head_tail[0]
          filename= head_tail[1] 
          archieveDir = dirname+'/'+ dt_string 
          if not os.path.exists(archieveDir):
            os.mkdir( archieveDir)
          archieveFile = archieveDir + '/'+ filename
          os.rename(path, archieveFile)


In [8]:
pd.read_sql('select institution, count(*) from syncCheck group by institution ', con)

,institution,count(*)
0,AUC,142894
1,CU,93400
2,ROSS,146636


In [9]:
#pd.read_sql('select * from cmitablelist', con)

In [49]:
#cmi=pd.read_excel('/content/drive/My Drive/Colab Notebooks/Sasi-Dev/CMITables.xlsx', sheet_name='Sheet1', header=0, names=['Institution','TableName'])

In [ ]:
#cmi.head(3)

In [51]:
cmi.to_sql(name='cmitablelist', con=con, if_exists='replace')

In [13]:
sql = 'select sc.runTimeStamp, sc.institution, sc."table" ,  sc.scount sourceCount, sc.traget SnowFlakeCount, (sc.traget - sc.scount) as Diff , (((sc.traget - sc.scount )/sc.scount) * 100) as Percent \
from cmitablelist c,  syncCheck sc \
where \
c.TablesNames = sc."table" AND \
c.Institution = sc.institution AND \
(sc.traget - sc.scount ) <> 0 \
order by sc.runTimeStamp, sc.institution;'

In [14]:
df_res=pd.read_sql(sql, con)

In [15]:
df_res= df_res.drop_duplicates()

In [16]:
df_resAUC=df_res[df_res.institution=='AUC']
df_resCU=df_res[df_res.institution=='CU']
df_resROSS=df_res[df_res.institution=='ROSS']


In [17]:
df_AUCvarience = df_resAUC.pivot(index='table',columns='runTimeStamp', values='Diff').reset_index()
df_CUvarience = df_resCU.pivot(index='table',columns='runTimeStamp', values='Diff').reset_index()
df_ROSSvarience = df_resROSS.pivot(index='table',columns='runTimeStamp', values='Diff').reset_index()

In [18]:
fl= '/content/drive/My Drive/Reports/'+dt_string+'_'+ 'CMIVarienace.xlsx'
with pd.ExcelWriter(fl) as writer:  
    df_AUCvarience.to_excel(writer, sheet_name='AUC')
    df_CUvarience.to_excel(writer, sheet_name='CU')
    df_ROSSvarience.to_excel(writer, sheet_name='ROSS')

In [19]:
Acols=df_AUCvarience.columns[-3:].insert(0,'table')
l_auc = df_AUCvarience[Acols]

Ccols=df_CUvarience.columns[-3:].insert(0,'table')
l_cu = df_CUvarience[Ccols]

Rcols=df_ROSSvarience.columns[-3:].insert(0,'table')
l_ross = df_ROSSvarience[Rcols]

In [20]:
fl3= '/content/drive/My Drive/Reports/'+dt_string+'_last3_CMIVarienace.xlsx'
with pd.ExcelWriter(fl3) as writer:  
    l_auc.to_excel(writer, sheet_name='AUC')
    l_cu.to_excel(writer, sheet_name='CU')
    l_ross.to_excel(writer, sheet_name='ROSS')

In [21]:
sqlx = 'select  sc.institution, sc."table" ,  sc.scount sourceCount, sc.traget SnowFlakeCount, (sc.traget - sc.scount) \
as Diff , ABS(((sc.traget - sc.scount )/sc.scount) * 100) as "Varience %" \
from cmitablelist c,  syncCheck sc \
where \
c.TablesNames = sc."table" AND \
c.Institution = sc.institution AND \
(sc.traget - sc.scount ) <> 0 AND \
sc.runTimeStamp in ( \
  select max(runTimeStamp)from syncCheck \
    group by institution ) \
order by  sc.institution;' 

In [22]:
df_rep=pd.read_sql(sqlx, con)
df_rep= df_rep.drop_duplicates()

In [23]:
df_rep['sourceCount']= df_rep['sourceCount'].apply(int)
df_rep['SnowFlakeCount']= df_rep['SnowFlakeCount'].apply(int)
df_rep['Diff']= df_rep['Diff'].apply(int)
df_rep['Varience %']= df_rep['Varience %'].round(6)

In [24]:
sqlx2 = 'Select institution, max(runTimeStamp) as  Dttime from syncCheck \
    group by institution '

In [25]:
df_idt=pd.read_sql(sqlx2, con)

In [26]:
dt= df_idt['Dttime'].max()
dts=pd.to_datetime(dt).strftime("%Y%m%dT%H%M%S")
sheetx =pd.to_datetime(dt).strftime("%m-%d-%Y_%H.%M")

In [27]:
df_idt

,institution,Dttime
0,AUC,08-26-20 08:03:56
1,CU,08-26-20 07:15:54
2,ROSS,08-26-20 07:31:43


In [28]:
xlfile = '/content/drive/My Drive/Reports/' +dts+'_CMIBNR_DecrepancyReport.xlsx'
#xlfile

In [29]:
 df_rep.to_excel(xlfile, sheet_name=sheetx)

In [30]:
con.close()

In [31]:
receiver_email= ['Christian.duque@adtalem.com',\
'Sanjay.patel@adtalem.com',\
'Deepthi.janapareddy@adtalem.com',\
'Kalai.vijaianand@adtalem.com',\
'Meenal.jain@adtalem.com',\
'Veerendra.ponduri@adtalem.com',\
'Sri.hari@adtalem.com',\
'ACE_Requests@adtalem.com', \
'WBehrns@aucmed.edu',\
'RByard@RossU.edu',\
'DKheterpal@chamberlain.edu',\
'AShamsuddin@chamberlain.edu',\
'Stephen.Haworth@adtalem.com',\
'Lucas.DeBritoRocha@adtalem.com',\
'Carlos.PiresJunior@adtalem.com', \
'sasikumar.manickam@adtalem.com',\
'Banipal.Michel@adtalem.com',\
'Eva.Pagacs@adtalem.com',\
'Ramakrishnan.Purushothaman@adtalem.com',\
'Eleftherios.Sigalos@adtalem.com']

In [32]:
#receiver_email

In [33]:
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

subject = "CMI Banner - Snowflake Discrepancy report  " + dts
body = "CMI: Banner - Snowflake Discrepancy report as of "+ sheetx

sender_email = "atgeanalytics@gmail.com"
password = "uzdkzrovvaefwnis"

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = ', '.join(receiver_email)
message["Subject"] = subject
#message["Bcc"] = receiver_email  # Recommended for mass emails

# Add body to email
message.attach(MIMEText(body, "plain"))

filename = xlfile  # In same directory as script
filenm = dts+'_CMIBNR_DecrepancyReport.xlsx'
# Open PDF file in binary mode
with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filenm}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)

In [54]:
#con = sqlite3.connect("/content/drive/My Drive/Colab Notebooks/Sasi-Dev/CMIAdminLog.db")

In [ ]:
#sqlx3="select * FROM 'cmitablelist' where TableName ='SPRCMNT'"
#pd.read_sql(sqlx3, con)


In [57]:
#con.close()